In [4]:
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from textblob import TextBlob
import statsmodels.api as sm

In [5]:
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


# ------------------------------------------Polarity as a factor--------------------------------------

### Sentences with positive meaning have positive polarity and negative meaning ones have negative polarity and neutral ones have zero as polarity

In [7]:
dfpol=pd.read_csv("C:/Users/user/Downloads/weibo_polarity.csv")

In [8]:
dfpol.head(10)

,Unnamed: 0,u_id,m_id,forward_count,comment_count,like_count,content,date,time,content_media_count,...,like_median,like_mean,Unnamed: 0.1,content_spchar,non_emoji_content,en_content,Unnamed: 1,url_rem,contentwurl,polarity
0,0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,0,0,0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,2015-02-23,17:41:29,0.0,...,0,0,0,['(' ')' '#' '#' '#' '#' '#' '#' '#' ...,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,Lijiang Tourism (sz002033) # ## stock stocks F...,NaN,NaN,Lijiang Tourism (sz002033) # ## stock stocks F...,-0.166667
1,1,d38e9bed5d98110dc2489d0d1cac3c2a,00755196c77936bf44656ada98291c59,0,0,0,想开了就是幸福，想不开就是痛苦…http://t.cn/RLqzYa1,2015-07-13,19:24:50,1.0,...,0,0,1,['#' '#' '，' '。' '@' '，' '！' '，' '╮' ...,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...,Chen Ling Ding # # red envelopes to make money...,NaN,NaN,Chen Ling Ding # # red envelopes to make money...,0.175000
2,2,d38e9bed5d98110dc2489d0d1cac3c2a,4fedf3888b1e16592f0e0bdc8b393845,0,0,0,300419浩丰科技#股票##股神##股市##炒股##财经##理财##投资# 股票庄家，要求...,2015-03-14,21:22:57,0.0,...,0,0,2,['。' '。' '。' '~' '，'],淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了,Taobao these sucker. . . Industrial gas fire n...,NaN,NaN,Taobao these sucker. . . Industrial gas fire n...,-0.400000
3,3,d38e9bed5d98110dc2489d0d1cac3c2a,91be0b8612265aae32725cd4fa80b222,0,0,0,女性被人占便宜从来不生气，是什么心态？http://t.cn/R2n2pRz,2015-06-18,16:25:36,1.0,...,0,0,3,['，' '[' ']'],看点不能说的，你们都懂[笑cry],"Aspect can not say, you know everything [laugh...",NaN,NaN,"Aspect can not say, you know everything [laugh...",0.000000
4,4,d38e9bed5d98110dc2489d0d1cac3c2a,bd2af99ecf1298f5539f0ddfcdd3ed64,0,0,0,网宿科技(sz300017)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,2015-02-23,17:35:31,0.0,...,0,0,4,[],111多张,Over 111 Zhang,NaN,NaN,Over 111 Zhang,0.000000
5,5,d38e9bed5d98110dc2489d0d1cac3c2a,182078c5a409834f2128b3c9c2c289c3,0,0,0,中国平安(sh601318)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,2015-02-23,17:30:13,0.0,...,0,0,5,['！' '！' '！' '@' ':' '/' '/' '.' '/'],有生之年！我最喜欢的up主跟我的三体勾搭到一起了！幸福感爆棚！ @黑桐谷歌 http://...,Lifetime! My favorite up with the main body ho...,NaN,NaN,Lifetime! My favorite up with the main body ho...,0.354167
6,6,d38e9bed5d98110dc2489d0d1cac3c2a,2c9697e5d6f1d9d479540173c4c374cb,0,0,0,300013新宁物流#股票##股神##股市##炒股##财经##理财##投资# 股票庄家，要求...,2015-03-14,21:24:36,0.0,...,0,0,6,['？' ':' '/' '/' '.' '/' '（' '@' '）'],论优衣库试衣间隔音效果好坏？ http://t.cn/RL5aSzp（分享自 @知乎）,On Uniqlo dressing room sound insulation is go...,NaN,NaN,On Uniqlo dressing room sound insulation is go...,0.133333
7,7,d38e9bed5d98110dc2489d0d1cac3c2a,0ce5d103d7712b398ee2e81f83f49751,0,0,0,今年最流行的空气感刘海，你剪了？（多图）http://t.cn/R2mdwd7,2015-06-19,22:31:51,1.0,...,0,0,7,['，' '。' '，' '，' '。'],如此平凡的日常一幕，还能够再积累多少呢。 终有一天，当我们到了看着这张照片能感受到一阵怀念的...,"So ordinary everyday scene, but also how much ...",NaN,NaN,"So ordinary everyday scene, but also how much ...",-0.083333
8,8,d38e9bed5d98110dc2489d0d1cac3c2a,a651facd0523d2a85a0717b83928c6c8,0,0,0,002326永太科技#股票##股神##股市##炒股##财经##理财##投资#推荐包赢股，盈利...,2015-03-05,16:36:16,0.0,...,0,0,8,['#' '#' '，' '，' '！' '@' '！' '，' '~' ...,#罗永浩的红包#二十三，糖瓜儿粘，抢个红包乐翻天！我抢到了罗永浩 和@_王先森就是我 一起发...,"Overheating of red # # xxiii, Tanggua children...",NaN,NaN,"Overheating of red # # xxiii, Tanggua children...",0.110937
9,9,d38e9bed5d98110dc2489d0d1cac3c2a,3e1895f6017e0214f7392013552ac96a,0,0,0,603636南威软件#股票##股神##股市##炒股##财经##理财##投资##新股##大盘#...,2015-03-23,12:30:09,0.0,...,0,0,9,['！' '，' '，' '？' '：' ':' '/' '/' '.' ...,有好东西分享给你！闪记笔记记事，最好用的中文待办软件，还等什么？快去下载： http://t...,There are good things to share with you! Flash...,NaN,NaN,There are good things to share with you! Flash...,0.625000


In [9]:
dfpol.columns

Index(['Unnamed: 0', 'u_id', 'm_id', 'forward_count', 'comment_count',
       'like_count', 'content', 'date', 'time', 'content_media_count',
       'content_#_count', 'content_@_count', 'content_?_count',
       'content_!_count', 'content_length', 'content_emoji_count', 'hour',
       'min', 'sec', 'forward_min', 'forward_max', 'forward_median',
       'forward_mean', 'comment_min', 'comment_max', 'comment_median',
       'comment_mean', 'like_min', 'like_max', 'like_median', 'like_mean',
       'Unnamed: 0.1', 'content_spchar', 'non_emoji_content', 'en_content',
       'Unnamed: 1', 'url_rem', 'contentwurl', 'polarity'],
      dtype='object')

In [10]:
dfpol['date']=pd.to_datetime(dfpol['date'],errors='coerce')
train_month=[g for n, g in dfpol.groupby(pd.Grouper(key='date',freq='M'))]

In [11]:
train_month[0]=pd.read_csv("C:/Users/user/Downloads/weibo_train_feb_cpts10000.csv")
train_month[1]=pd.read_csv("C:/Users/user/Downloads/weibo_train_march_cpts10000.csv")
train_month[2]=pd.read_csv("C:/Users/user/Downloads/weibo_train_april_cpts10000.csv")
train_month[3]=pd.read_csv("C:/Users/user/Downloads/weibo_train_may_cpts10000.csv")
train_month[4]=pd.read_csv("C:/Users/user/Downloads/weibo_train_june_cpts10000.csv")
train_month[5]=pd.read_csv("C:/Users/user/Downloads/weibo_train_july_cpts10000.csv")


In [12]:
frames1=[train_month[0],train_month[1],train_month[2],train_month[3],train_month[4]]
train=pd.concat(frames1)
predict=train_month[5]

## Model 7: (Factors: Media, Length, Emoji, Median,Polarity)

In [9]:
X_train1=train[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
lm1=linear_model.LinearRegression()
model1=lm1.fit(X_train1,Y_train1)
pred1=lm1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [11]:
print(model1.coef_)
print(model1.intercept_)

[[ -3.60345081e-01   6.74510609e-03  -9.03871388e+00  -5.69877934e+00
    1.46618263e+01  -3.00702607e-01]
 [ -4.52559256e-01   2.88854313e-04  -2.58935392e+00   3.20973326e-01
    2.56664104e+00   1.78063868e-01]
 [ -1.70607807e-01  -1.73336140e-03  -2.49319057e+00  -7.66383744e-01
    3.84877692e+00   1.34944972e-01]]
[ 0.09961692  0.27754213  0.23556883]


In [12]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result51.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result51.csv")

In [13]:
print(mean_squared_error(Y_test1,result1))

21.2745912995


In [14]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Score:35.39%


## Analysis: Result with Polarity as factor are satisfactory considering the data used for train. This might prove to be a good factor for whole dataset prediction.

# Model 8: (Factors: Media, Length,Median,Polarity)

In [15]:
X_train1=train[["content_media_count","content_length","forward_mean","comment_mean","like_mean","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_mean","comment_mean","like_mean","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
lm1=linear_model.LinearRegression()
model1=lm1.fit(X_train1,Y_train1)
pred1=lm1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [17]:
print(model1.coef_)
print(model1.intercept_)

[[  1.84153944e-01   1.16499672e-02   1.23769730e+00   5.03985664e-01
   -5.30125012e-01   2.40824642e-01]
 [ -4.34205066e-01   2.65588900e-03   4.65693627e-02   8.21537197e-01
   -4.63148333e-03   8.38895495e-02]
 [ -1.07997853e-01  -6.08462330e-04   1.24279057e-01  -2.25635181e-02
    7.47271891e-01   1.46482193e-01]]
[-1.685958    0.04881615  0.00796132]


In [18]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result51.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result51.csv")

In [19]:
print(mean_squared_error(Y_test1,result1))

21.0421169299


In [20]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:34.58%


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

# Model 9: (Factors: Media, Length, Emoji, Median,Mean,Polarity)

In [28]:
X_train1=train[["content_media_count","content_length","forward_mean","forward_median","comment_mean","comment_median","like_mean","like_median","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_mean","forward_median","comment_mean","comment_median","like_mean","like_median","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [29]:
lm1=linear_model.LinearRegression()
model1=lm1.fit(X_train1,Y_train1)
pred1=lm1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [30]:
print(model1.coef_)
print(model1.intercept_)

[[  4.24002976e-01   1.39830856e-02   1.77218462e+00   5.28953775e+00
    5.32073405e-01   2.72797599e+00   9.08606236e-02  -8.60889360e+00
    4.57317719e-01]
 [ -3.94970258e-01   2.85864515e-03   1.34655712e-01   5.14556038e-01
    7.61232739e-01   4.96108919e-01   2.61640134e-02  -1.03251918e+00
    1.30303697e-01]
 [ -2.52656284e-01  -1.69108114e-03  -1.95657316e-01  -2.51251894e+00
    9.14012006e-02  -1.71242671e+00   4.71094851e-01   4.48754073e+00
   -7.29299942e-04]]
[-2.47378947 -0.06752787  0.45657126]


In [31]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result51.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result51.csv")

In [32]:
print(mean_squared_error(Y_test1,result1))

20.9952895539


In [33]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:34.22%


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

# Model10: (Factors: Media, Length, Emoji, Median,Polarity) with OLS

### OLS is a type of linear least sqaures methods for estimating parameters in a linear regression model

In [13]:
X_train1=train[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
model1=sm.OLS(Y_train1,X_train1).fit()
pred1=model1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [16]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result52.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result52.csv")

In [17]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Score:35.39%


# Model11: (Factors: Media, Length, Emoji, Median,Polarity) with Ridge regression

### Ridge regression is used to prevent multicollinearity among variables by shrinking the parameters

In [18]:
X_train1=train[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
lm1=linear_model.Ridge(alpha=3)
model1=lm1.fit(X_train1,Y_train1)
pred1=lm1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [20]:
print(model1.coef_)
print(model1.intercept_)

[[ -3.86510208e-01   6.41054941e-03  -8.34912342e+00  -5.36638332e+00
    1.39393510e+01  -2.63639792e-01]
 [ -4.56306722e-01   1.98158232e-04  -2.45402009e+00   3.77596491e-01
    2.43239453e+00   1.84454477e-01]
 [ -1.76866329e-01  -1.82770299e-03  -2.31640230e+00  -6.84262708e-01
    3.66606306e+00   1.43628492e-01]]
[ 0.12872141  0.28577674  0.2444117 ]


In [21]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result53.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result53.csv")

In [22]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:35.48%


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

# Model 12: (Factors: Media, Length, Emoji, Median,Polarity) with Lasso regression

### Lasso regression does automatic feature selection that means if some features are correlated then lasso will pick only one feature

In [23]:
X_train1=train[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_train1=train[["forward_count","comment_count","like_count"]]
X_test1=predict[["content_media_count","content_length","forward_median","comment_median","like_median","polarity"]]
Y_test1=predict[["forward_count","comment_count","like_count"]]

pd.options.mode.use_inf_as_na = True
X_train1.fillna(X_train1.max(),inplace=True)
X_test1.fillna(X_test1.max(),inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [24]:
lm1=Lasso(alpha=0.01)
model1=lm1.fit(X_train1,Y_train1)
pred1=lm1.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [25]:
print(model1.coef_)
print(model1.intercept_)

[[ -3.93885876e-01   5.30762986e-03  -7.01981756e+00  -4.71960564e+00
    1.25490926e+01  -8.17241037e-02]
 [ -3.98749279e-01  -1.15532108e-03  -1.29700987e+00   7.65902250e-01
    1.36959059e+00   8.67988313e-02]
 [ -1.33179860e-01  -2.99999206e-03  -8.19004244e-01  -0.00000000e+00
    2.14881763e+00   4.81255664e-02]]
[ 0.09786835  0.4897481   0.43776902]


In [26]:
np.savetxt("C:/Users/user/Downloads/weibo_predict_result54.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result1=pd.read_csv("C:/Users/user/Downloads/weibo_predict_result54.csv")

In [27]:
train_real_pred=Y_test1
train_real_pred['fp']=result1['forward_count']
train_real_pred['cp']=result1['comment_count']
train_real_pred['lp']=result1['like_count']
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

Score:35.58%


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 